# Validation of islanded mode equivalence

## Import and helpers
Import core packages for the toy model.

In [1]:
import logging
import numpy as np
import pandas as pd
import pypsa
import plotly.graph_objects as go

logger = logging.getLogger(__name__)

Helpers

In [2]:
def extract_capacity(n: pypsa.Network) -> pd.DataFrame:
    """Extract a tidy table of optimal capacities for all extendable assets."""
    df = n.statistics.optimal_capacity().reset_index()
    df.columns = ["component", "name", "capacity"]
    return df


def extract_energy_balance(n: pypsa.Network) -> pd.DataFrame:
    """Extract a tidy table of energy balance statistics."""
    df = n.statistics.energy_balance()
    df.rename("energy", inplace=True)
    return df.reset_index()


def calculate_total_costs(n: pypsa.Network, exclude_grid: bool = False) -> float:
    """Calculate total costs (capex + opex), optionally excluding AC lines and DC links."""
    costs = pd.concat([n.statistics.capex(), n.statistics.opex()], axis=1, keys=["capex", "opex"])
    if exclude_grid:
        costs = costs[~(
            ((costs.index.get_level_values("component") == "Line") &
             (costs.index.get_level_values("carrier") == "AC")) |
            ((costs.index.get_level_values("component") == "Link") &
             (costs.index.get_level_values("carrier") == "DC"))
        )]
    return costs.fillna(0).sum().sum()


def create_bar_plot(pivot_df, x_col, y_cols, labels, title, yaxis_title):
    """Create a grouped bar plot with multiple scenarios."""
    fig = go.Figure()
    for y_col, label in zip(y_cols, labels):
        fig.add_trace(go.Bar(x=pivot_df[x_col], y=pivot_df[y_col], name=label))
    fig.update_layout(
        title=title,
        xaxis_title=x_col.title(),
        yaxis_title=yaxis_title,
        barmode="group",
        hovermode="x unified",
        template="plotly_white",
    )
    return fig

## Data prep
Load the base network, adjust loads, and add load-shedding generators.

In [3]:
n_elec_path = "data/networks/base_s_50_elec_.nc"
solver_name = "gurobi"
solver_options = {"OutputFlag": 0}  # minimize solver log output

# Load network
n = pypsa.Network(n_elec_path)

# Base styling and carriers
n.carriers.loc["", "color"] = "#aaaaaa"
n.add("Carrier", "load shedding", color="#aa0000")

# Increase load
n.loads_t.p_set *= 1.5

# Add load-shedding generators
spatial_nodes = n.buses.index.tolist()
ls_names = [f"{bus} load shedding" for bus in spatial_nodes]

n.add(
    "Generator",
    pd.Index(ls_names),
    bus=pd.Series(spatial_nodes, index=ls_names),
    p_nom_extendable=True,
    p_nom_max=np.inf,
    capital_cost=0.1,
    marginal_cost=10000.0,
    carrier="load shedding",
)


INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.4)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, carriers, generators, lines, links, loads, storage_units, stores, sub_networks


## Optimisation
Prepare scenario copies for baseline and islanded configurations. Run optimisations for baseline, s_max_pu = 0 / p_max_pu = 0, and fully islanded cases.

In [4]:
print("Running baseline optimisation...")
n1 = n.copy()
n1.optimize(solver_name=solver_name, solver_options=solver_options)

cap1 = extract_capacity(n1)
eb1 = extract_energy_balance(n1)

Running baseline optimisation...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 588.11it/s]
INFO:linopy.io: Writing time: 0.28s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.06 seconds


INFO:gurobipy:Reading time = 0.06 seconds


obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 7.66e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [5]:
print("Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...")
n2 = n.copy()
if not n.lines.empty:
    n2.lines["s_max_pu"] = 0
if not n2.links.empty:
    dc_links = n2.links[n2.links.carrier == "DC"].index
    n2.links.loc[dc_links, ["p_max_pu", "p_min_pu"]] = 0

n2.optimize(solver_name=solver_name, solver_options=solver_options)

cap2 = extract_capacity(n2)
eb2 = extract_energy_balance(n2)

Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 430.12it/s]
INFO:linopy.io: Writing time: 0.28s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.07 seconds


INFO:gurobipy:Reading time = 0.07 seconds


obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 9.80e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [6]:
print("Running islanded optimisation (lines and links removed)...")
n3 = n.copy()
if not n.lines.empty:
    n3.remove("Line", n3.lines.index)
if not n3.links.empty:
    n3.remove("Link", n3.links[n3.links.carrier == "DC"].index)

n3.optimize(solver_name=solver_name, solver_options=solver_options)

cap3 = extract_capacity(n3)
eb3 = extract_energy_balance(n3)

Running islanded optimisation (lines and links removed)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 12/12 [00:00<00:00, 520.49it/s]
INFO:linopy.io: Writing time: 0.25s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.05 seconds


INFO:gurobipy:Reading time = 0.05 seconds


obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 21798 primals, 44499 duals
Objective: 9.80e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


## Metrics
Metrics for validation

In [7]:
total_costs = pd.DataFrame({
    "scenario": ["grid", "maxpu0", "removed"],
    "total_cost": [
        np.round(calculate_total_costs(n1) / 1e9, 3),
        np.round(calculate_total_costs(n2, exclude_grid=True) / 1e9, 3),
        np.round(calculate_total_costs(n3, exclude_grid=True) / 1e9, 3),
    ],
})
print("\nTotal annual costs by scenario:")
print(total_costs.to_string(index=False))

mp_diff = n3.buses_t.marginal_price - n2.buses_t.marginal_price
mp_diff_mean = mp_diff.abs().mean()
print("\nMean absolute marginal price difference:")
print(mp_diff_mean.head())


Total annual costs by scenario:
scenario  total_cost
    grid     304.914
  maxpu0     317.299
 removed     317.299

Mean absolute marginal price difference:
name
AL0 0    0.000000e+00
AT0 0    0.000000e+00
BA0 0    0.000000e+00
BE0 0    0.000000e+00
BG0 0    4.144833e-15
dtype: float64


## Plots
Optimal capacities and energy balances across scenarios.

In [8]:
capacities = pd.concat([
    cap1.assign(scenario="grid"),
    cap2.assign(scenario="maxpu0"),
    cap3.assign(scenario="removed"),
], ignore_index=True)
capacities_pivot = capacities.pivot_table(
    index=["component", "name"],
    columns="scenario",
    values="capacity",
    fill_value=0,
).reset_index()

fig_cap = create_bar_plot(
    capacities_pivot,
    "name",
    ["grid", "maxpu0", "removed"],
    ["Grid exists", "Grid removed (s_max_pu=0)", "Grid removed (lines removed)"],
    "Optimal capacity",
    "Installed Capacity [MW]",
)
fig_cap.show()

In [9]:
energy_balances = pd.concat([
    eb1.assign(scenario="grid"),
    eb2.assign(scenario="maxpu0"),
    eb3.assign(scenario="removed"),
], ignore_index=True)
energy_balances_pivot = energy_balances.pivot_table(
    index="carrier",
    columns="scenario",
    values="energy",
    fill_value=0,
).reset_index()

fig_eb = create_bar_plot(
    energy_balances_pivot,
    "carrier",
    ["grid", "maxpu0", "removed"],
    ["Grid exists", "Grid removed (s_max_pu=0)", "Grid removed (lines removed)"],
    "Energy balance",
    "Energy [MWh]",
)
fig_eb.show()